1.	Download the labeled dataset, if you like use a dummy email address for registration. There must be 8 data files, each representing a particular cyber-attack type and it's day, and it's collected pcap data. 

![Screenshot 1](https://i.ibb.co/Fgc8TQ2/Picture1.png)

2. Pick one of the data files, call it Dataset 1, and examine its features. Make sure it has more than one class value for its label. 

In [1]:
import pandas as pd

dataset1 = pd.read_csv('GeneratedLabelledFlows\TrafficLabelling\Wednesday-workingHours.pcap_ISCX.csv')

dataset1.head()

####################################
#######FIXED
####################################
print(dataset1[' Label'].unique())


['BENIGN' 'DoS slowloris' 'DoS Slowhttptest' 'DoS Hulk' 'DoS GoldenEye'
 'Heartbleed']


3. For the Dataset 1, pick a machine learning methodology and justify your choice. It contains labeled data, so it’s a supervised learning problem. 



        Classification is the optimal optizion sice its a supervised learning problem with los of features, hence GaussianNB as a model would work better with this amount of data.

4. Process the class feature/category as binary classes for supervised learning, assign BENIGN to value 0 and the rest to value 1. Check its balance for the Dataset 1. 

In [2]:
##############################
#FIXED
################################


import numpy as np

def convert_y(y):
    bin_y = []
    for i in y:
        if str(i) == "BENIGN":
            bin_y.append(0)
        else:
            bin_y.append(1)
    return bin_y

def Average(lst):
    return sum(lst) / len(lst)

datasets = ['GeneratedLabelledFlows\TrafficLabelling\Monday-WorkingHours.pcap_ISCX.csv',
'GeneratedLabelledFlows\TrafficLabelling\Tuesday-WorkingHours.pcap_ISCX.csv',
'GeneratedLabelledFlows\TrafficLabelling\Wednesday-workingHours.pcap_ISCX.csv', 
'GeneratedLabelledFlows\TrafficLabelling\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', 
'GeneratedLabelledFlows\TrafficLabelling\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', 
'GeneratedLabelledFlows\TrafficLabelling\Friday-WorkingHours-Morning.pcap_ISCX.csv', 
'GeneratedLabelledFlows\TrafficLabelling\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', 
'GeneratedLabelledFlows\TrafficLabelling\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv']

datasets_pandas = []

for d in datasets:
    curr_d = pd.read_csv(d)
    y = curr_d.iloc[:,-1].values
    bin_y = convert_y(y)
    curr_d['Label'] = bin_y
    class_balance = (curr_d['Label'] / 
                  curr_d['Label'].sum()) * 100
    average = Average(class_balance)
    print(f'Class balance {round(average * 100, 2)}%')
    datasets_pandas.append(curr_d)

Class balance nan%
Class balance 0.02%
Class balance 0.01%
Class balance 0.06%
Class balance 0.03%
Class balance 0.05%
Class balance 0.03%
Class balance 0.04%


5. Explore Dataset 1 features with respect to the class. (Hint: features Source Port and Destination Port are very useful, research and find out important networking port numbers and one-hot-encode them. Unimportant port numbers or source port numbers can be assigned to a feature called 'other ports') 

        The most important features are Source IP, Source Port, Destination IP, Destination Port and Protocol. Followed by the timestamp, which is crucial for packet capture and analysis, Flow Duration which is the amount of time communication between two endpoints (information rate per unit time) and the totals in forward and backward packets and their lengths. 



6. Display some histograms and anything you deem fit to pick independent Dataset 1 features. (Hint: source/destination bytes, packets, ports and the duration features) 

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#Using Pearson Correlation
cor = datasets_pandas[2].corr()
cor.style.background_gradient(cmap='coolwarm').set_precision(2)

C:\Users\ddpir\AppData\Local\Temp/ipykernel_13064/50927445.py:7: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  cor.style.background_gradient(cmap='coolwarm').set_precision(2)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\style.py:2813: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\style.py:2814: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


In [4]:
#Correlation with output variable
cor_target = cor["Label"] #Selecting highly correlated features

#positive correlation - when value increases, output increases
positive_relevant_features = cor_target[cor_target>0.5]

#some negastive correlation - when value decreases, output decreases
negative_relevant_features = cor_target[cor_target<-0.25]

#get selected features column names into list
selected_features = positive_relevant_features.index.tolist() + negative_relevant_features.index.tolist()
selected_features.remove('Label')


selected_features

['Bwd Packet Length Max',
 ' Bwd Packet Length Mean',
 ' Bwd Packet Length Std',
 ' Flow IAT Std',
 ' Flow IAT Max',
 ' Fwd IAT Std',
 ' Fwd IAT Max',
 ' Max Packet Length',
 ' Packet Length Mean',
 ' Packet Length Std',
 ' Packet Length Variance',
 ' Average Packet Size',
 ' Avg Bwd Segment Size',
 'Idle Mean',
 ' Idle Max',
 ' Idle Min',
 ' Destination Port',
 ' Protocol',
 ' Bwd Packet Length Min',
 ' Min Packet Length',
 ' Down/Up Ratio']

7. Attempt a few classifier models and report their 10-fold CV performance.

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
le = LabelEncoder()
from sklearn.metrics import accuracy_score
import numpy as np

def encode_category(le, df, column):
    df[column] = le.fit_transform(df[column])
    return df


results = []
def eval_classifier(_clf, _X, _y):
    acc = []
    kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
    for train_index, test_index in kf.split(_X, _y):
        _clf.fit(_X[train_index], _y[train_index].ravel())
        y_pred = _clf.predict(_X[test_index])
        acc += [accuracy_score(_y[test_index], y_pred)]
    r = np.array(acc).mean()
    r = np.round(r, decimals = 2)
    results.append(r)
    
#getting sample from dataset
sample = datasets_pandas[2].sample(frac=0.1, random_state=1)

for feature in selected_features:
    encode_category(le, sample, feature)

X = sample.loc[:, selected_features].values
y = sample.loc[:, ['Label']].values

eval_classifier(GaussianNB(), X, y)
eval_classifier(RandomForestClassifier(), X, y)
eval_classifier(KNeighborsClassifier(), X, y)

print('Gaussian NB')
print(results[0])

print('Random Forest')
print(results[1])

print('K Neighbors')
print(results[2])

Gaussian NB
0.84
Random Forest
1.0
K Neighbors
0.99


8. Convert your code to be used for the remaining 7 datasets, i.e. Datasets 2-8. 

9. Pick a classifier algorithm and report it's evaluation for the remaining 7 datasets. Note that one dataset has a single class, which might need an unsupervised learning. 

Answer for question 8 and 9


In [6]:
X = []
y = []
found = False

def encode_category(le, df, column):
    df[column] = le.fit_transform(df[column])
    return df

def cor_factor(_cor_target, sign):
    
    features = pd.Series()
    if sign == '+': 
        factor = 0.5
        while features.shape[0] < 1:
            features = cor_target[cor_target>factor]
            factor = factor - 0.1
            
    else:
        factor = -0.9
        while features.shape[0] < 1:
            features = cor_target[cor_target>factor]
            factor = factor + 0.1
        return factor
    



def get_x_y(df):
    
    cor = df.corr()
    cor_target = cor["Label"] #Selecting highly correlated features
    
    
    #pos_factor = cor_factor(cor_target, '+')
    positive_relevant_features = cor_target[cor_target>0.1]
    
    #neg_factor = cor_factor(cor_target, '-')
    negative_relevant_features = cor_target[cor_target<-0.5]
    
    selected_features = positive_relevant_features.index.tolist() + negative_relevant_features.index.tolist()
    if 'Label' in selected_features:
        selected_features.remove('Label')
    
    sample = df.sample(frac=0.1, random_state=1)
    
    for feature in selected_features:
        encode_category(le, sample, feature)
    
    X = sample.loc[:, selected_features].values
    y = sample.loc[:, ['Label']].values


    

def eval_classifier(_clf, _X, _y):
    acc = []

    kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
    
    for train_index, test_index in kf.split(_X, _y):
        _clf.fit(_X[train_index], _y[train_index].ravel())
        y_pred = _clf.predict(_X[test_index])
        acc += [accuracy_score(_y[test_index], y_pred)]
    r = np.array(acc).mean()
    r = np.round(r, decimals = 2)
    return r

for d in datasets_pandas:

    cor = d.corr()
    cor_target = cor["Label"] #Selecting highly correlated features
    
    
    #pos_factor = cor_factor(cor_target, '+')
    positive_relevant_features = cor_target[cor_target>0.1]
    
    #neg_factor = cor_factor(cor_target, '-')
    negative_relevant_features = cor_target[cor_target<-0.1]
    
    selected_features = positive_relevant_features.index.tolist() + negative_relevant_features.index.tolist()
    if 'Label' in selected_features:
        selected_features.remove('Label')
    
    sample = d.sample(frac=0.1, random_state=1)
    
    for feature in selected_features:
        encode_category(le, sample, feature)
    
    X = sample.loc[:, selected_features].values
    y = sample.loc[:, ['Label']].values
    
    if X.shape[1] > 0 and y.shape[1] > 0:
        curr_result = eval_classifier(GaussianNB(), X, y)
        print(f'10 K-Fold Evaluation={curr_result}')
        

10 K-Fold Evaluation=0.5
10 K-Fold Evaluation=0.86


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


10 K-Fold Evaluation=0.98
10 K-Fold Evaluation=0.51
10 K-Fold Evaluation=0.98
10 K-Fold Evaluation=0.99


10. Briefly write up your thoughts about developing a machine learning model where you are not a subject matter expert, such as, developing a cybersecurity intrusion detection pipeline as in this assignment. 


Developing this machine learning model was valuable in understanding the basics of how internet protocols work and communicate with each other. It was interesting to see in between the many packages sent, backwarded and their length, which ones the correlation matrix picked. Solely knowing the ip address of the sender and receiver is not enough to study the data, knowing all the stops along the way and the details of the packets opens a new way for machine learning and network analysis.